# DynamoDB interactive tool

Choose a mode:

* Migrate: Read from one table and write to another table
* Only read: Read from a table and save it to a CSV
* Only write: Write to a table from a CSV

In [ ]:
# Standard library
from dataclasses import dataclass
from typing import Optional
from io import StringIO

# 3rd party
import pandas as pd
from pandas import DataFrame

# Local
from utils import Singleton
from core import FILTER_DOCS_LINK, AppState, run, get_credentials
from client import ALL_REGIONS, DEFAULT_REGION, DEFAULT_TABLE_NAME, Environment

PROFILES = get_credentials()
STATE = AppState()


In [ ]:
"""See https://stackoverflow.com/a/61077368/4809575"""
import sys

from IPython import get_ipython

RUNTIME_REF = get_ipython()


def hide_traceback(exc_tuple=None, filename=None, tb_offset=None, exception_only=False, running_compiled_code=False):
    """
    To debug, comment the next line assignment. 
    Use 'print()' and 'display()' instead of 'log.info()'
    """
    etype, value, tb = sys.exc_info()
    return RUNTIME_REF._showtraceback(etype, value, RUNTIME_REF.InteractiveTB.get_exception_only(etype, value))


RUNTIME_REF.showtraceback = hide_traceback


In [ ]:
import json
from subprocess import Popen

from IPython.display import display
from ipywidgets import FileUpload, Output, Button, Textarea, Layout

from core import read_sheet

output_widget = Output()  # ? used as a console.log and to display CSVs

with open('./filter.json', 'r') as f:
    DEFAULT_FILTER = json.load(f)


def on_csv_output_click(*args):
    with output_widget:
        output_widget.clear_output()

        if STATE.saved_filename is not None:
            dataframe = read_sheet(STATE.saved_filename)
            display(dataframe)
        else:
            display('There is no data retrieved from DynamoDB. No data to display')


def on_csv_open_click(*args):
    if STATE.saved_filename is not None:
        Popen(['open', STATE.saved_filename])


def on_file_upload(file):
    if len(file) > 0:
        (file_data, ) = file.new
        in_memory_file = pd.ExcelFile(file_data.content.tobytes())
        STATE.input_data = pd.read_excel(in_memory_file)

        with output_widget:        
            output_widget.clear_output()
            display(STATE.input_data)
    else:
        display('Uploaded file is empty')

        
def on_filter_update(properties):
    STATE.input_filter = properties['new'] or {}


csv_output_button_widget = Button(
    description='Preview data',
    tooltip='Opens a preview of the downloaded data',
    disabled=True,
    icon='file-spreadsheet')

csv_output_button_widget.on_click(on_csv_output_click)

csv_open_button_widget = Button(
    description='Open in Editor',
    tooltip='Opens the downloaded data in a tabular editor',
    disabled=True,
    icon='file-spreadsheet')

csv_open_button_widget.on_click(on_csv_open_click)

input_filter_widget = Textarea(
    value=str({}),
    placeholder=json.dumps(DEFAULT_FILTER, indent=4),
    disabled=False,
    layout=Layout(height="auto", width="auto")
)

input_filter_widget.observe(on_filter_update, names='value')

upload_widget = FileUpload(accept='.xlsx', description='Upload', multiple=False)
upload_widget.observe(on_file_upload, names='value')


In [ ]:
from ipywidgets import interact


interact_manual = interact.options(manual=True, manual_name="Run")


@interact_manual(
    continuous_update=False,
    # Default values
    only_read=False,
    input_profile=PROFILES,
    input_region=ALL_REGIONS,
    input_table_name=DEFAULT_TABLE_NAME,
    only_write=False,
    output_profile=PROFILES,
    output_region=ALL_REGIONS,
    output_table_name=DEFAULT_TABLE_NAME,
    dry_run=True
)
def main(only_read, input_profile, input_region, input_table_name, only_write, output_profile,
         output_region, output_table_name, dry_run):

    STATE.dry_run = dry_run
    csv_output_button_widget.disabled = True
    csv_open_button_widget.disabled = True

    if only_read:
        # ? Use case 1: read from a table & keep the data locally
        STATE.saved_filename = None
        STATE.input_data = None
        STATE.input_environment = Environment(input_profile, input_region, input_table_name)
        STATE.output_environment = None
    elif only_write:
        # ? Use case 2: read from a local CSV file & write to a table
        if not STATE.input_data is not None:
            raise ValueError('Please upload a file')

        STATE.saved_filename = None
        STATE.input_environment = None
        STATE.output_environment = Environment(output_profile, output_region, output_table_name)
    else:
        # ? Use case 3: read from a table & write to another table
        STATE.input_data = None
        STATE.saved_filename = None
        STATE.input_environment = Environment(input_profile, input_region, input_table_name)
        STATE.output_environment = Environment(output_profile, output_region, output_table_name)

    saved_filename = run(STATE)
    upload_widget.value = []
    print(saved_filename)

    if saved_filename is not None:
        csv_output_button_widget.disabled = False
        csv_open_button_widget.disabled = False
        STATE.saved_filename = saved_filename


In [ ]:
from ipywidgets import GridspecLayout, HTML

buttons_layout = GridspecLayout(1, 2)
buttons_layout[0, 0] = csv_output_button_widget
buttons_layout[0, 1] = csv_open_button_widget

additional_layouts = GridspecLayout(4, 4, height='auto', width='auto', align_items='center')
additional_layouts[0, 0] = buttons_layout
additional_layouts[1, 0] = upload_widget
additional_layouts[2, 0] = HTML(f'<p>Filter (please <a href={FILTER_DOCS_LINK}"><u>see docs</u></a>)</p>')
additional_layouts[3, 0] = input_filter_widget
additional_layouts[:, 1] = output_widget
additional_layouts


In [ ]:
# from boto3 import Session
# from client import _deserialize

# session = Session(profile_name='805789757283_PA_DEVELOPER', region_name='eu-west-1')
# client = session.client('dynamodb')
# res = session.resource('dynamodb')

# table_name = 'Configuration-vrewwqazvrfzrigag63c7doeqy-int'
# table = res.Table(table_name)
# print(table)

# data = client.scan(TableName=table_name, **{})

# print(len(data['Items']))
# print(len([_deserialize(_) for _ in data['Items']]))